## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database_copy.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,80.82,71,29,10.13,scattered clouds
1,1,Chuy,UY,-33.6971,-53.4616,73.76,61,90,20.38,overcast clouds
2,2,Lompoc,US,34.6391,-120.4579,54.00,66,1,10.11,clear sky
3,3,Viedma,AR,-40.8135,-62.9967,91.00,20,0,2.17,clear sky
4,4,Tiksi,RU,71.6872,128.8694,-24.77,86,98,3.42,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,80.82,71,29,10.13,scattered clouds
5,5,Belmonte,BR,-15.8631,-38.8828,84.20,74,20,10.36,few clouds
10,10,Bambous Virieux,MU,-20.3428,57.7575,80.60,74,20,9.22,few clouds
20,20,Rikitea,PF,-23.1203,-134.9692,78.21,69,100,10.87,overcast clouds
21,21,Grand Baie,MU,-20.0182,57.5802,80.01,75,1,5.99,light rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Country                199
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,80.82,scattered clouds,-24.7911,-50.0119,
5,Belmonte,BR,84.20,few clouds,-15.8631,-38.8828,
10,Bambous Virieux,MU,80.60,few clouds,-20.3428,57.7575,
20,Rikitea,PF,78.21,overcast clouds,-23.1203,-134.9692,
21,Grand Baie,MU,80.01,light rain,-20.0182,57.5802,
31,Avarua,CK,75.20,few clouds,-21.2078,-159.7750,
32,Georgetown,MY,81.00,few clouds,5.4112,100.3354,
33,Hambantota,LK,77.23,overcast clouds,6.1241,81.1185,
35,Samarai,PG,80.64,clear sky,-10.6167,150.6667,
37,Oistins,BB,84.20,scattered clouds,13.0667,-59.5333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        continue
        

In [16]:

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,80.82,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY
5,Belmonte,BR,84.20,few clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
10,Bambous Virieux,MU,80.60,few clouds,-20.3428,57.7575,Casa Tia Villa
20,Rikitea,PF,78.21,overcast clouds,-23.1203,-134.9692,Pension Maro'i
21,Grand Baie,MU,80.01,light rain,-20.0182,57.5802,Trou aux Biches Beachcomber Golf Resort & Spa


In [17]:
# 7. Drop the rows where there is no Hotel Name.
len(hotel_df[hotel_df["Hotel Name"] == ''])
drop_list = hotel_df[hotel_df["Hotel Name"]==''].index
hotel_df.drop(index=drop_list,inplace=True)

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))